In [1]:
# 文本通常只是数据集中的字符串，但并非所有字符串特征都应该被当做文本来处理，常见四种类型
# 分类数据、可以在语义上映射为类别的自由字符串、结构化字符串数据、文本数据
# 在文本分析的语境中，数据集通常被称为语料库，每个由单个文本表示的数据点被称为文档

In [2]:
# 示例应用：电影评论的情感分析
# 使用aclimdb数据集，数据集包含评论文本，及一个标签用于表示该评论是“正面的”还是“负面的”
from sklearn.datasets import load_files

# load_files返回一个Bunch对象，其中包含训练文本和训练标签
reviews_train = load_files("./data/aclimdb/train/")
text_trian, y_train = reviews_train.data, reviews_train.target
print("tpye of text_train:{}".format(type(text_trian)))
print("length of text_train:{}".format(len(text_trian)))
print("text_train[1]:\n{}".format(text_trian[1]))

tpye of text_train:<class 'list'>
length of text_train:75000
text_train[1]:
b"Amount of disappointment I am getting these days seeing movies like Partner, Jhoom Barabar and now, Heyy Babyy is gonna end my habit of seeing first day shows.<br /><br />The movie is an utter disappointment because it had the potential to become a laugh riot only if the d\xc3\xa9butant director, Sajid Khan hadn't tried too many things. Only saving grace in the movie were the last thirty minutes, which were seriously funny elsewhere the movie fails miserably. First half was desperately been tried to look funny but wasn't. Next 45 minutes were emotional and looked totally artificial and illogical.<br /><br />OK, when you are out for a movie like this you don't expect much logic but all the flaws tend to appear when you don't enjoy the movie and thats the case with Heyy Babyy. Acting is good but thats not enough to keep one interested.<br /><br />For the positives, you can take hot actresses, last 30 minutes, s

In [4]:
# 清洗文本中存在的HTML换行符
text_trian = [doc.replace(b"<br />", b" ") for doc in text_trian]

In [6]:
import numpy as np

print("Samples per class(traing):{}".format(np.bincount(y_train)))

Samples per class(traing):[12500 12500 50000]


array([0, 1, 2])

numpy.ndarray